# Importing Packages

In [1]:
from itertools import product
from Bio import SeqIO
from sklearn.model_selection import train_test_split
from sklearn import metrics
import sklearn.ensemble
from Bio.Seq import Seq
from Bio.Alphabet import generic_dna


# Generating combinations

In [2]:
a=list(product(['A','G','C','T'],repeat = 4))
res = [''.join(tups) for tups in a] 
print(len(res))

256


In [3]:

c=[]
for i in res:
    a = Seq(i, generic_dna)  
    b=a.reverse_complement()
    if b not in c:
        c.append(a)

# Calculating the frequency of 4 letter-subsequences

In [4]:
def calculate_frequency(seq):
    
    frequency = {}
    for combination in c:
        frequency[combination] = 0 
    for i in range(len(seq) - 4):  
        combination = seq[i:i+4]
        if (combination in frequency):
            frequency[combination] = frequency[combination] + 1 
        rev_com_combinations = str(Seq(combination).reverse_complement())
        if (rev_com_combinations in frequency):
            frequency[rev_com_combinations] = frequency[rev_com_combinations] + 1
    ret_arr = []
    for key in frequency:
        ret_arr.append( float(float(frequency[key]) / 1500.0 ))
    return ret_arr

# Reading the Data

In [5]:
vista_data=[]
random_data=[]
chr21_data=[]

for line in SeqIO.parse("C:\\Users\\aravi\\Desktop\\Desktop\\Bioinformatics-project4-master\\Bioinformatics-project4-master\\vista1500", "fasta"):
    vista_data.append(str(line.seq).upper())

for line in SeqIO.parse("C:\\Users\\aravi\\Desktop\\Desktop\\Bioinformatics-project4-master\\Bioinformatics-project4-master\\randoms1500", "fasta"):
    random_data.append(str(line.seq).upper())

for letter in SeqIO.parse("C:\\Users\\aravi\\Desktop\\Desktop\\Bioinformatics-project4-master\\Bioinformatics-project4-master\\chr21.fa", "fasta"):
    chr21_data.append(str(letter.seq).upper())

In [6]:

data=chr21_data[0]
x_chr21 = []
contains_N = []
for i in range(0, len(data) - 1500, 750):
    sub_seq = data[i:i+1500]
    

    #if (sub_seq.find('n') == -1):
    if 'N' not in sub_seq:
        x_chr21.append(calculate_frequency(sub_seq))
        contains_N.append(0)
    else:
        contains_N.append(1)

c:\users\aravi\appdata\local\programs\python\python37\lib\site-packages\Bio\Seq.py:182: BiopythonWarning: Biopython Seq objects now use string comparison. Older versions of Biopython used object comparison. During this transition, please use hash(id(my_seq)) or my_dict[id(my_seq)] if you want the old behaviour, or use hash(str(my_seq)) or my_dict[str(my_seq)] for the new string hashing behaviour.
  BiopythonWarning,


In [7]:
len(chr21_data[0])

46709983

# Preparing Data for training

In [8]:
x = []
y = []

for line in vista_data:
    x.append(calculate_frequency(line))
    y.append(1)
for line in random_data:
    x.append(calculate_frequency(line))
    y.append(0)

In [9]:
# 80% for trainig and 20% for testing
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

# Initializing the RandomForestClassifier

In [10]:
clf = sklearn.ensemble.RandomForestClassifier(n_estimators=5000)

# Fitting the model

In [11]:
clf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5000,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

# Make Predictions

In [12]:
y_test_pred=clf.predict(x_test)
y_train_pred=clf.predict(x_train)

In [13]:
print("Test Accuracy:",metrics.accuracy_score(y_test, y_test_pred))
print("Train Accuracy:",metrics.accuracy_score(y_train, y_train_pred))

Test Accuracy: 0.7837259100642399
Train Accuracy: 1.0


In [14]:

predictions = clf.predict_proba(x_chr21)

In [15]:
i = 0
ret = []
p = [prediction[1] for prediction in predictions]
mean = float(sum(p) / len(p))
for has_N in contains_N:

    if (has_N == 0):
        ret.append(p[i])
        i = i + 1
    else:
        ret.append(mean)

# Saving the data to wig file

In [16]:
with open("chr21.wig", "w") as f:
    f.write("fixedStep chrom=chr21 start=0 step=750 span=1500\n")
    f.write("\n".join([str(c) for c in ret]))